# RAG with Multi Data Source

## Wikipedia Tools

In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [3]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)
print(wiki.name)

wikipedia


In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectore_db=FAISS.from_documents(documents,OllamaEmbeddings(model="llama2"))
retriever=vectore_db.as_retriever
retriever

In [ ]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool=create_retriever_tool(retriever,"langsmith_search","Search fro information about langsmith")
retriever_tool.name

## ArXiv Tools

In [6]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
print(arxiv.name)

arxiv


In [ ]:
tools=[wiki,arxiv,retriever_tool]


In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()

from langchain_community.llms import Ollama

llm=Ollama(model="llama2")



In [24]:
from langchain import hub
prompt=hub.pull("yeyuan/llama2-finetune-template")
prompt.name

In [ ]:
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

In [ ]:
agent

In [ ]:
# Agent Executer
from langchain.agents import AgentExecutor
agent_execitor=AgentExecutor(agent=agent,tools=tools,verbose=True)

In [ ]:
agent_execitor.invoke({"input":"tell me about langchain?"})